# Upload training file and necessary preprocessing

In [ ]:
from google.colab import files
uploaded = files.upload()

## Pre-Requisits

In [ ]:
# !pip install langdetect
!pip install numpy==1.23.5
!pip install -q -U "tensorflow-text==2.11.*"
!pip install -q tf-models-official==2.11.0

In [ ]:
# from langdetect import detect
import pandas as pd
import numpy as np

training_set = pd.read_json("EXIST2023_training.json")
dev_set = pd.read_json("EXIST2023_dev.json")
test_set = pd.read_json("EXIST2023_test_clean.json")

## Preparing the dataset 

In [ ]:
tweets_d = []
labels_d = []
sexism_d = []
tLabels_d = []
sLabels_d = []
otherLang_d = 0
english_d = 0
id_d = []
sexId_d = []

for i in dev_set:  
  countY = 0
  countN = 0

  # if detect(df[i].tweet) == 'en':
  #   english +=1
  # else:
  #   otherLang +=1
  #   continue

  if (i < 200000) or (i >= 203260 and i <=400001) :
    continue
  id_d.append(int(i))
  # print(detect(df[i].tweet))
  tweets_d.append(dev_set[i].tweet)

  for l in dev_set[i].labels_task1:
    if l == 'YES':
      countY +=1
    if l == 'NO':
      countN +=1
  if countY == countN :
    labels_d.append(-1)
  elif countY > countN:
    labels_d.append(1)
    sexId_d.append(i)
    sexism_d.append(dev_set[i].tweet)
    tune = {
        "REPORTED": 0, "JUDGEMENTAL": 0, "DIRECT": 0
    }
    sense = {
        "MISOGYNY-NON-SEXUAL-VIOLENCE":0,
        "IDEOLOGICAL-INEQUALITY": 0, 
        "STEREOTYPING-DOMINANCE": 0, 
        "OBJECTIFICATION": 0, 
        "SEXUAL-VIOLENCE":0, 
        "-1":0
    }
    for m in dev_set[i].labels_task2:
      if m == 'REPORTED':
        tune["REPORTED"] +=1
      if m == 'JUDGEMENTAL':
        tune["JUDGEMENTAL"] +=1
      if m == 'DIRECT':
        tune["DIRECT"] +=1 
    tLabels_d.append(max(tune, key=tune.get))

    for n in dev_set[i].labels_task3:

      if n[0] == 'MISOGYNY-NON-SEXUAL-VIOLENCE':
        sense["MISOGYNY-NON-SEXUAL-VIOLENCE"] +=1
      if n[0] == 'IDEOLOGICAL-INEQUALITY':
        sense["IDEOLOGICAL-INEQUALITY"] +=1
      if n[0] == 'STEREOTYPING-DOMINANCE':
        sense["STEREOTYPING-DOMINANCE"] +=1
      if n[0] == 'OBJECTIFICATION':
        sense["OBJECTIFICATION"] +=1
      if n[0] == 'SEXUAL-VIOLENCE':
        sense["SEXUAL-VIOLENCE"] +=1
      if n[0] == '-':
        sense["-1"] +=1

    sLabels_d.append(max(sense, key=sense.get))
  else:
    labels_d.append(0)



In [ ]:
tweets = []
labels = []
sexism = []
tLabels = []
sLabels = []
otherLang = 0
english = 0
id = []
sexId = []

for i in training_set :  
  countY = 0
  countN = 0

  # if detect(df[i].tweet) == 'en':
  #   english +=1
  # else:
  #   otherLang +=1
  #   continue

  if (i < 200000) or (i >= 203260 and i <=400001) :
    continue
  id.append(int(i))
  # print(detect(df[i].tweet))
  tweets.append(training_set[i].tweet)


  for l in training_set[i].labels_task1:
    if l == 'YES':
      countY +=1
    if l == 'NO':
      countN +=1
  if countY == countN :
    labels.append(-1)
  elif countY > countN:
    labels.append(1)
    sexId.append(i)
    sexism.append(training_set[i].tweet)
    tune = {
        "REPORTED": 0, "JUDGEMENTAL": 0, "DIRECT": 0
    }
    sense = {
        "MISOGYNY-NON-SEXUAL-VIOLENCE":0,
        "IDEOLOGICAL-INEQUALITY": 0, 
        "STEREOTYPING-DOMINANCE": 0, 
        "OBJECTIFICATION": 0, 
        "SEXUAL-VIOLENCE":0, 
        "-1":0
    }
    for m in training_set[i].labels_task2:
      if m == 'REPORTED':
        tune["REPORTED"] +=1
      if m == 'JUDGEMENTAL':
        tune["JUDGEMENTAL"] +=1
      if m == 'DIRECT':
        tune["DIRECT"] +=1 
    tLabels.append(max(tune, key=tune.get))

    for n in training_set[i].labels_task3:

      if n[0] == 'MISOGYNY-NON-SEXUAL-VIOLENCE':
        sense["MISOGYNY-NON-SEXUAL-VIOLENCE"] +=1
      if n[0] == 'IDEOLOGICAL-INEQUALITY':
        sense["IDEOLOGICAL-INEQUALITY"] +=1
      if n[0] == 'STEREOTYPING-DOMINANCE':
        sense["STEREOTYPING-DOMINANCE"] +=1
      if n[0] == 'OBJECTIFICATION':
        sense["OBJECTIFICATION"] +=1
      if n[0] == 'SEXUAL-VIOLENCE':
        sense["SEXUAL-VIOLENCE"] +=1
      if n[0] == '-':
        sense["-1"] +=1

    sLabels.append(max(sense, key=sense.get))
  else:
    labels.append(0)


# print("English tweets ", english)
# print("Other tweets ", otherLang)

In [ ]:
test_tweets = []
tweet_id_t = []
for i in test_set:
  if i < 600000:
    continue
  tweet_id_t.append(i)
  test_tweets.append(test_set[i].tweet)

## Put the dataset into dataframe

In [ ]:
df_tweets = pd.DataFrame(
    {'id': id,
     'tweets': tweets,
    })

df_sexism = pd.DataFrame(
    {'id': sexId,
     'sexism': sexism,
    })

dev_tweets = pd.DataFrame( 
    {
        'id': id_d,
     'tweets': tweets_d,
    })

dev_sexism = pd.DataFrame(
    {'id': sexId_d,
     'sexism': sexism_d,
    })

test_set = pd.DataFrame(
    {
        "id": tweet_id_t,
        "tweet": test_tweets
    }
)


## Preprocessing the Tweets

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

## 1. Removal of punctuation and capitlization
## 2. Tokenizing
## 3. Removal of stopwords
## 4. Stemming

stopwords = nltk.corpus.stopwords.words("english")

#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

def data_cleansing(tweet):  
    
    # removal of extra spaces
    re_pattern = re.compile(r'\s+')
    spaceRefined = tweet.str.replace(re_pattern, ' ')

    # removal of @name[mention]
    re_pattern = re.compile(r'@[\w\-]+')
    usernameRefined = spaceRefined.str.replace(re_pattern, '')

    # removal of links[https://abc.com]
    re_pattern =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = usernameRefined.str.replace(re_pattern, '')
    
    # removal of punctuations and numbers
    puncRefined = tweets.str.replace("[^a-zA-Z]", " ")
    # remove whitespace with a single space
    newtweet = puncRefined.str.replace(r'\s+', ' ')
    # remove leading and trailing whitespace
    newtweet = newtweet.str.replace(r'^\s+|\s+?$','')
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    # removal of capitalization
    tweet_lower = newtweet.str.lower()
    
    # tokenizing
    tokenized_tweet = tweet_lower.apply(lambda x: x.split())
    
    # removal of stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [item for item in x if item not in stopwords])
    
    # stemming of the tweets
    tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 
    
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tweets_p= tokenized_tweet
    
    return tweets_p

In [ ]:
refinedTweets = data_cleansing(df_tweets.tweets)   
df_tweets['processed_tweets'] = refinedTweets
print(df_tweets[["tweets","processed_tweets"]].head(10))

In [ ]:
refinedTweets = data_cleansing(dev_tweets.tweets)   
dev_tweets['processed_tweets'] = refinedTweets
print(dev_tweets[["tweets","processed_tweets"]].head(10))

In [ ]:
refinedTweets = data_cleansing(df_sexism.sexism)   
df_sexism['processed_tweets'] = refinedTweets
print(df_sexism[["sexism","processed_tweets"]].head(10))

In [ ]:
refinedTweets = data_cleansing(dev_sexism.sexism)   
dev_sexism['processed_tweets'] = refinedTweets
print(dev_sexism[["sexism","processed_tweets"]].head(10))

In [ ]:
refinedTweets = data_cleansing(test_set.tweet)   
test_set['processed_tweets'] = refinedTweets
print(test_set[["tweet","processed_tweets"]].head(10))

## Putting the labels into dataframe

In [ ]:
df_sexism['label'] = sLabels
df_tweets['label'] = labels
dev_sexism['label'] = sLabels_d
dev_tweets['label'] = labels_d


# Visualization of all tweets in the dataset and Sexism Tweets

In [ ]:
# visualizing which of the word is most commonly used in the dataset
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

all_words = ' '.join([word for word in df_tweets['processed_tweets'] ])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

#random=0.30
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# visualizing which of the word is most commonly used in the dataset
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

all_words = ' '.join([word for word in df_sexism['processed_tweets'] ])
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

#random=0.30
plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

# Preparing dataset labels for Machine Learning Algorithms

## Data Labelling

In [ ]:
df_tweets = df_tweets[df_tweets['label'] != -1]
df_tweets['label'].value_counts()

In [ ]:
dev_tweets = dev_tweets[dev_tweets['label'] != -1]
dev_tweets['label'].value_counts()

In [ ]:
df_sexism = df_sexism[df_sexism['label'] != "-1"]
df_sexism['label'].value_counts()

In [ ]:
dev_sexism = dev_sexism[dev_sexism['label'] != "-1"]
dev_sexism['label'].value_counts()

In [ ]:
df_sexism['label']=df_sexism['label'].replace('IDEOLOGICAL-INEQUALITY', 0)
df_sexism['label']=df_sexism['label'].replace('STEREOTYPING-DOMINANCE', 1)
df_sexism['label']=df_sexism['label'].replace('OBJECTIFICATION', 2)
df_sexism['label']=df_sexism['label'].replace('MISOGYNY-NON-SEXUAL-VIOLENCE', 3)
df_sexism['label']=df_sexism['label'].replace('SEXUAL-VIOLENCE', 4)
df_sexism['label'].value_counts()

In [ ]:
dev_sexism['label']=dev_sexism['label'].replace('IDEOLOGICAL-INEQUALITY', 0)
dev_sexism['label']=dev_sexism['label'].replace('STEREOTYPING-DOMINANCE', 1)
dev_sexism['label']=dev_sexism['label'].replace('OBJECTIFICATION', 2)
dev_sexism['label']=dev_sexism['label'].replace('MISOGYNY-NON-SEXUAL-VIOLENCE', 3)
dev_sexism['label']=dev_sexism['label'].replace('SEXUAL-VIOLENCE', 4)
dev_sexism['label'].value_counts()

## Word Embedding using TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)
tfidf = tfidf_vectorizer.fit_transform(df_tweets['processed_tweets'])

print(tfidf.shape)

X = tfidf
y = df_tweets['label'].astype(int)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

tfidf_vectorizer_dev = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)
tfidf_dev = tfidf_vectorizer_dev.fit_transform(dev_tweets['processed_tweets'])

print(tfidf_dev.shape)

X_dev = tfidf_dev
y_dev = dev_tweets['label'].astype(int)
X_dev_train_tfidf, X_dev_test_tfidf, y_dev_train, y_dev_test = train_test_split(X_dev, y_dev, random_state=42, test_size=0.1)

## Hyperparameter tuning of Random Forest

### Hyperparameter Tuning- RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
random_search = RandomizedSearchCV(RandomForestClassifier(),
                                   random_grid)
random_search.fit(X_dev_train_tfidf, y_dev_train)

In [ ]:
print(random_search.best_estimator_)

I used these hyperparameters in the section random forest with hyperparameter tuning below in task 1

# Task 1: Sexism Detection

### Random Forest ( By Default Hyper Parameter Tuning )

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


rf=RandomForestClassifier()
rf.fit(X_train_tfidf,y_train)
y_preds = rf.predict(X_test_tfidf)
# acc1=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
# print("Random Forest, Accuracy Score:",acc1)
rfAccuracy = 100 * accuracy_score(y_test, y_preds)

## Random Forest ( With Hyperparameter tuning using Random-Search method) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf1=RandomForestClassifier(bootstrap=False, max_depth=80, max_features='auto',
                       min_samples_leaf=4, n_estimators=1800)
rf1.fit(X_train_tfidf,y_train)
y_preds = rf1.predict(X_test_tfidf)
# acc1=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
# print("Random Forest, Accuracy Score:",acc1)

## Feeding Dataset to Bert Model


In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

from sklearn.model_selection import train_test_split
X = np.array(df_tweets.tweets)
y = np.array(df_tweets['label'].astype(int))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')



In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(X_test))
# print(tf.sigmoid(bert_raw_result))
tf.keras.utils.plot_model(classifier_model)

# Initializing the loss
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.binary_accuracy

# Epochs Setting
epochs = 5
steps_per_epoch = 10
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr, num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps, optimizer_type='adamw')

In [ ]:
classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
print(f'Training model with {tfhub_handle_encoder}')

# Uncomment the line below for training purpose
# history = classifier_model.fit(X_train,y_train,epochs=epochs)
loss, accuracy = classifier_model.evaluate(X_test, y_test)

In [ ]:
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

## Comparsion b/w Random Forest and Bert

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# creating the dataset
data = {'RandomForest':rfAccuracy, 'Bert': accuracy * 100, }
courses = list(data.keys())
values = list(data.values())

fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(courses, values, color ='maroon',
		width = 0.4)

plt.xlabel("Algorithm Comparison")
plt.ylabel("Accuracy")
plt.title("Sexism Classification using Random Forest and Bert")
plt.show()


## Result Generation

In [ ]:
sexismCategorization = []

result_Task1_Both_Labels = [] 
for id, tweets in test_set.iterrows():
  Test = tfidf_vectorizer.transform(np.array([str(tweets['processed_tweets'])]))
  hardLabel = rf.predict(Test)
  softLabel = rf.predict_proba(Test)
  
  if hardLabel[0] == 1:
    sexismCategorization.append([tweets.id, tweets['processed_tweets']])

  sub = {
      str(tweets.id):
      {
        "Hard Label": "Yes" if hardLabel[0] == 1 else "NO",
        "Soft Label": {"Yes":softLabel[0][1], "No": softLabel[0][0] }
      }
  }
  result_Task1_Both_Labels.append(sub)

In [ ]:
import json

with open('Test_set Task 1.json', 'w') as f:
  json.dump(result_Task1_Both_Labels ,f)


### Storing the Trained Model

In [ ]:
import pickle
pickle.dump(rf, open('Trained_Model_Task1.pkl', 'wb'))

### Loading the Stored Model

In [ ]:
pickled_model = pickle.load(open('Trained_Model_Task1.pkl', 'rb'))
y_preds = pickled_model.predict(X_test_tfidf)
report = classification_report( y_test, y_preds )
print(report)

# Task 3: Sexism Categorization


## Tf-Idf Conversion

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)
tfidf = tfidf_vectorizer.fit_transform(df_sexism['processed_tweets'])

print(tfidf.shape)

X = tfidf
y = df_sexism['label'].astype(int)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

tfidf_vectorizer_dev = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)
tfidf_dev = tfidf_vectorizer_dev.fit_transform(dev_sexism['processed_tweets'])

print(tfidf_dev.shape)

X_d = tfidf_dev
y_d = dev_sexism['label'].astype(int)
X_dev_train_tfidf, X_dev_test_tfidf, y_dev_train, y_dev_test = train_test_split(X_d, y_d, random_state=42, test_size=0.1)

In [ ]:
# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression().fit(X_train_tfidf,y_train)
# y_preds = model.predict(X_test_tfidf)
# report = classification_report( y_test, y_preds )

# print(report)

## Hyperparameter Tuning of Random Forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

### Hyperparameter Tuning -RandomizedSearchCV

In [ ]:
random_search = RandomizedSearchCV(RandomForestClassifier(),
                                   random_grid)
random_search.fit(X_dev_train_tfidf, y_dev_train)

I used these parameter in the section Random-Forest-with Hyperparameter tuning

In [ ]:
print(random_search.best_estimator_)

## Random Forest ( By default Hyperparameter Tuning)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf=RandomForestClassifier()
rf.fit(X_train_tfidf,y_train)
y_preds = rf.predict(X_test_tfidf)
# acc1=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
# print("Random Forest, Accuracy Score:",acc1)

rfAccuracy1 = 100 * accuracy_score(y_test, y_preds)

## Random Forest ( With Hyperparameter Tuning ) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf1=RandomForestClassifier(bootstrap=False, max_depth=40, max_features='auto',
                       min_samples_leaf=4, n_estimators=1600)
rf1.fit(X_train_tfidf,y_train)
y_preds = rf1.predict(X_test_tfidf)
# acc1=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
# print("Random Forest, Accuracy Score:",acc1)

## Feeding Sexism Categorization to Bert

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

from sklearn.model_selection import train_test_split
X = np.array(df_sexism.sexism)
y = np.array(df_sexism['label'].astype(int))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
# classifier_model = build_classifier_model()
# bert_raw_result = classifier_model(tf.constant(X_test))
# print(tf.sigmoid(bert_raw_result))
tf.keras.utils.plot_model(classifier_model)

# Initializing the loss
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.binary_accuracy

# Epochs Setting
epochs = 5
steps_per_epoch = 10
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr, num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps, optimizer_type='adamw')

In [ ]:
classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
print(f'Training model with {tfhub_handle_encoder}')

history = classifier_model.fit(X_train, y_train, epochs=2, batch_size=32)
loss1, accuracy1 = classifier_model.evaluate(X_test, y_test)


In [ ]:
print(f'Loss: {loss1}')
print(f'Accuracy: {accuracy1}')

## Comparsion between Random Forest and Bert Task 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# creating the dataset
data = {'RandomForest':rfAccuracy1, 'Bert':accuracy1 * 100, }
courses = list(data.keys())
values = list(data.values())

fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(courses, values, color ='maroon',
		width = 0.4)

plt.xlabel("Algorithm Comparison")
plt.ylabel("Accuracy")
plt.title("Sexism Categorization using Random forest and Bert")
plt.show()


## Generating Results

### Storing the Trained Model

In [ ]:
import pickle
pickle.dump(rf, open('Trained_Model_task3.pkl', 'wb'))

### Loading the trained Model

In [ ]:
pickled_model = pickle.load(open('Trained_Model_task3.pkl', 'rb'))
y_preds = pickled_model.predict(X_test_tfidf)
report = classification_report( y_test, y_preds )
print(report)

In [ ]:
task3_Sexism_Category = list()

for id, tweet in sexismCategorization:
  Test = tfidf_vectorizer.transform(np.array([str(tweet)]))
  hardLabel = rf.predict(Test)
  softLabel = rf.predict_proba(Test)

  sub = {
      str(id):
      {
        "Hard Label": "IDEOLOGICAL-INEQUALITY" if hardLabel[0] == 0 else "STEREOTYPING-DOMINANCE" if hardLabel[0] == 1 else "OBJECTIFICATION" if hardLabel[0] == 2 else "MISOGYNY-NON-SEXUAL-VIOLENCE" if hardLabel[0] == 3 else "SEXUAL-VIOLENCE",
        "Soft Label": {"IDEOLOGICAL-INEQUALITY":round(softLabel[0][0],3), "STEREOTYPING-DOMINANCE": round(softLabel[0][1],3), "OBJECTIFICATION": round(softLabel[0][2],3), "MISOGYNY-NON-SEXUAL-VIOLENCE": round(softLabel[0][3],3), "SEXUAL-VIOLENCE": round(softLabel[0][3],3) }
      }
  }

  task3_Sexism_Category.append(sub)

  

In [ ]:
import json

with open('Task 3.json', 'w') as f:
  json.dump(task3_Sexism_Category ,f)